# Notebook 02: Pipeline RAG Completo - De Documentos a Respuestas Fundamentadas

---

## Objetivos de aprendizaje

Al finalizar este notebook, seras capaz de:

1. **Comprender** la arquitectura end-to-end de un pipeline RAG (Retrieval-Augmented Generation)
2. **Implementar** estrategias de chunking y evaluar sus trade-offs
3. **Crear embeddings** con OpenAI y almacenarlos en un vector store (ChromaDB)
4. **Construir** un sistema de retrieval con filtros por metadata
5. **Integrar** retrieval + generacion en un pipeline completo y funcional
6. **Evaluar** la calidad de las respuestas (faithfulness, relevance, groundedness)
7. **Comparar** respuestas de un LLM "naive" vs un LLM aumentado con RAG

### Prerequisitos
- Notebook 01 completado (conceptos basicos de embeddings y similitud)
- API key de OpenAI configurada en `.env`
- Familiaridad basica con Python y f-strings

### Tiempo estimado: 90-120 minutos

> **Nota importante**: Este notebook sigue el principio de *"show, don't tell"*. Cada concepto se introduce con una breve explicacion teorica seguida inmediatamente de codigo ejecutable. La mejor forma de aprender RAG es construyendo uno.

---

## Arquitectura RAG: Vista de Pajaro

Antes de escribir una sola linea de codigo, entendamos el flujo completo.

### Pipeline de Indexacion (offline, se ejecuta una vez)

```
┌─────────────┐     ┌──────────────┐     ┌──────────────┐     ┌──────────────────┐
│  DOCUMENTOS  │────>│   CHUNKING   │────>│  EMBEDDING   │────>│   VECTOR STORE   │
│  (.md files) │     │ (split text) │     │  (OpenAI)    │     │   (ChromaDB)     │
└─────────────┘     └──────────────┘     └──────────────┘     └──────────────────┘
                                                                       │
                                                                       │ persistido
                                                                       v
                                                               ┌──────────────────┐
                                                               │   Indices listos  │
                                                               │   para consulta   │
                                                               └──────────────────┘
```

### Pipeline de Consulta (online, se ejecuta por cada query del usuario)

```
┌──────────┐     ┌──────────────┐     ┌───────────────┐     ┌──────────────┐     ┌────────────┐
│  QUERY   │────>│  EMBEDDING   │────>│   RETRIEVAL   │────>│   CONTEXT    │────>│    LLM     │
│ (usuario)│     │  del query   │     │  (top-k mas   │     │  assembly    │     │  (OpenAI)  │
│          │     │              │     │   similares)  │     │              │     │            │
└──────────┘     └──────────────┘     └───────────────┘     └──────────────┘     └────────────┘
                                                                                       │
                                                                                       v
                                                                               ┌────────────┐
                                                                               │  RESPUESTA │
                                                                               │ fundamentada│
                                                                               └────────────┘
```

### Por que esta arquitectura?

| Componente | Funcion | Analogia |
|:-----------|:--------|:---------|
| Chunking | Dividir documentos en fragmentos manejables | Crear fichas de estudio a partir de un libro |
| Embedding | Convertir texto a vectores numericos | Traducir palabras a coordenadas en un mapa semantico |
| Vector Store | Almacenar y buscar vectores eficientemente | Una biblioteca con indice por temas |
| Retrieval | Encontrar los fragmentos mas relevantes | Un bibliotecario que sabe donde buscar |
| LLM + Context | Generar respuesta basada en evidencia | Un experto que responde citando fuentes |

---

## Setup e Importaciones

Instalamos y cargamos todas las dependencias necesarias.

In [ ]:
# === Instalacion de dependencias (ejecutar solo la primera vez) ===
# %pip install openai chromadb langchain langchain-text-splitters python-dotenv matplotlib

In [ ]:
# === Importaciones ===
import os
import json
import textwrap
from pathlib import Path
from typing import Optional

# OpenAI para embeddings y generacion
from openai import OpenAI

# ChromaDB como vector store
import chromadb
from chromadb.config import Settings

# LangChain para estrategias de chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Visualizacion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Variables de entorno
from dotenv import load_dotenv

# --- Cargar variables de entorno ---
load_dotenv()

# --- Inicializar cliente de OpenAI ---
client = OpenAI()  # Usa OPENAI_API_KEY del entorno automaticamente

# --- Verificacion ---
print("=" * 60)
print("SETUP COMPLETO")
print("=" * 60)
print(f"OpenAI API key cargada: {'Si' if os.getenv('OPENAI_API_KEY') else 'No'}")
print(f"ChromaDB version: {chromadb.__version__}")
print(f"Directorio de trabajo: {os.getcwd()}")
print("=" * 60)

---

## Paso 1: Carga de Documentos

Todo pipeline RAG comienza con **documentos fuente**. La calidad de tu RAG depende directamente de la calidad de estos documentos.

### Que hace un buen documento fuente?

- **Estructura clara**: Headers, secciones, listas. Facilita el chunking.
- **Informacion precisa y actualizada**: Garbage in, garbage out.
- **Metadata rica**: Fecha, autor, categoria. Permite filtros en retrieval.
- **Sin ambiguedades**: Informacion contradictoria confunde al retriever y al LLM.

Vamos a cargar dos bases de conocimiento de NovaTech Solutions:
1. `base_conocimiento_productos.md` - Informacion comercial (planes, precios, features)
2. `base_conocimiento_tecnica.md` - Informacion tecnica (arquitectura, deploys, troubleshooting)

In [ ]:
# === Paso 1: Cargar documentos fuente ===

# Ruta al directorio de datos (relativa al notebook)
DATA_DIR = Path("../data")

# Definimos los archivos a cargar con su metadata
archivos_fuente: dict[str, dict] = {
    "base_conocimiento_productos.md": {
        "domain": "productos",
        "description": "Informacion comercial de productos NovaTech"
    },
    "base_conocimiento_tecnica.md": {
        "domain": "tecnica",
        "description": "Guia tecnica y operaciones NovaTech"
    }
}

# Cargar contenido de cada archivo
documentos: dict[str, str] = {}

for nombre_archivo, metadata in archivos_fuente.items():
    ruta = DATA_DIR / nombre_archivo
    
    if not ruta.exists():
        print(f"ADVERTENCIA: No se encontro {ruta}")
        continue
    
    with open(ruta, "r", encoding="utf-8") as f:
        contenido = f.read()
    
    documentos[nombre_archivo] = contenido
    
    # Vista previa
    print(f"\n{'=' * 60}")
    print(f"Archivo: {nombre_archivo}")
    print(f"Dominio: {metadata['domain']}")
    print(f"Longitud: {len(contenido):,} caracteres | {len(contenido.split()):,} palabras")
    print(f"Lineas: {len(contenido.splitlines()):,}")
    print(f"{'-' * 60}")
    print(f"Preview (primeros 300 caracteres):")
    print(contenido[:300])
    print(f"{'=' * 60}")

print(f"\nTotal documentos cargados: {len(documentos)}")
print(f"Total caracteres: {sum(len(c) for c in documentos.values()):,}")

---

## Paso 2: Estrategias de Chunking

El **chunking** es una de las decisiones mas criticas en un pipeline RAG. Determina:

- **Granularidad del retrieval**: Chunks muy grandes traen ruido; muy pequenos pierden contexto.
- **Calidad del embedding**: Un embedding de un chunk coherente es mas util que uno de texto cortado arbitrariamente.
- **Costo**: Mas chunks = mas embeddings = mas tokens = mas dinero.

### Tres estrategias principales

| Estrategia | Descripcion | Pros | Contras |
|:-----------|:------------|:-----|:--------|
| **Fixed-Size** | Cortar cada N caracteres | Simple, predecible | Rompe oraciones y parrafos |
| **Recursive Character** | Intentar cortar por `\n\n`, luego `\n`, luego ` `, luego caracter | Respeta estructura | Chunks de tamano variable |
| **Semantic** | Agrupar por similitud semantica | Chunks mas coherentes | Mas costoso y complejo |

Implementaremos las dos primeras y las compararemos.

### 2a. Fixed-Size Chunking (implementacion manual)

In [ ]:
# === Estrategia 1: Fixed-Size Chunking ===
# Implementacion manual para entender exactamente que sucede

def fixed_chunks(text: str, size: int = 500, overlap: int = 50) -> list[str]:
    """
    Divide texto en chunks de tamano fijo con overlap.
    
    Args:
        text: Texto a dividir.
        size: Tamano de cada chunk en caracteres.
        overlap: Cantidad de caracteres que se solapan entre chunks consecutivos.
                 El overlap previene que informacion importante se pierda
                 en el corte entre dos chunks.
    
    Returns:
        Lista de strings, cada uno de longitud <= size.
    """
    if overlap >= size:
        raise ValueError(f"El overlap ({overlap}) debe ser menor que el size ({size})")
    
    chunks: list[str] = []
    inicio = 0
    
    while inicio < len(text):
        fin = inicio + size
        chunk = text[inicio:fin]
        chunks.append(chunk)
        inicio += size - overlap  # Avanzamos menos que el size para crear overlap
    
    return chunks


# --- Aplicar fixed chunking a ambos documentos ---
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

fixed_chunks_por_doc: dict[str, list[str]] = {}

for nombre, contenido in documentos.items():
    chunks = fixed_chunks(contenido, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP)
    fixed_chunks_por_doc[nombre] = chunks
    
    print(f"\n{'=' * 60}")
    print(f"Documento: {nombre}")
    print(f"Chunks generados: {len(chunks)}")
    print(f"Tamano promedio: {sum(len(c) for c in chunks) / len(chunks):.0f} caracteres")
    print(f"Tamano min/max: {min(len(c) for c in chunks)} / {max(len(c) for c in chunks)}")

# --- Mostrar un chunk de ejemplo ---
ejemplo_chunk = fixed_chunks_por_doc["base_conocimiento_productos.md"][2]
print(f"\n{'=' * 60}")
print(f"EJEMPLO - Chunk #2 de base_conocimiento_productos.md:")
print(f"{'=' * 60}")
print(ejemplo_chunk)
print(f"\n[Longitud: {len(ejemplo_chunk)} caracteres]")
print("\n>>> Observa: el corte puede ocurrir en medio de una oracion o seccion.")

### 2b. Recursive Character Splitting (LangChain)

In [ ]:
# === Estrategia 2: Recursive Character Splitting ===
# LangChain intenta dividir respetando la estructura del texto

# El splitter intenta dividir en este orden de prioridad:
# 1. "\n\n" (parrafos)
# 2. "\n" (lineas)
# 3. " " (palabras)
# 4. "" (caracteres individuales, ultimo recurso)

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Orden de prioridad
)

# --- Aplicar recursive splitting a ambos documentos ---
recursive_chunks_por_doc: dict[str, list[str]] = {}

for nombre, contenido in documentos.items():
    chunks = recursive_splitter.split_text(contenido)
    recursive_chunks_por_doc[nombre] = chunks
    
    print(f"\n{'=' * 60}")
    print(f"Documento: {nombre}")
    print(f"Chunks generados: {len(chunks)}")
    print(f"Tamano promedio: {sum(len(c) for c in chunks) / len(chunks):.0f} caracteres")
    print(f"Tamano min/max: {min(len(c) for c in chunks)} / {max(len(c) for c in chunks)}")

# --- Mostrar un chunk de ejemplo ---
ejemplo_recursive = recursive_chunks_por_doc["base_conocimiento_productos.md"][2]
print(f"\n{'=' * 60}")
print(f"EJEMPLO - Chunk #2 de base_conocimiento_productos.md (Recursive):")
print(f"{'=' * 60}")
print(ejemplo_recursive)
print(f"\n[Longitud: {len(ejemplo_recursive)} caracteres]")
print("\n>>> Observa: el recursive splitter intenta respetar limites de parrafo y linea.")

### 2c. Comparacion de Estrategias

In [ ]:
# === Comparacion lado a lado ===

print("COMPARACION DE ESTRATEGIAS DE CHUNKING")
print("=" * 70)
print(f"{'Metrica':<35} {'Fixed-Size':>15} {'Recursive':>15}")
print("-" * 70)

# Agregar todos los chunks de ambos documentos
all_fixed = [c for chunks in fixed_chunks_por_doc.values() for c in chunks]
all_recursive = [c for chunks in recursive_chunks_por_doc.values() for c in chunks]

metricas = [
    ("Total chunks", len(all_fixed), len(all_recursive)),
    ("Tamano promedio (chars)", 
     f"{sum(len(c) for c in all_fixed) / len(all_fixed):.0f}",
     f"{sum(len(c) for c in all_recursive) / len(all_recursive):.0f}"),
    ("Tamano min (chars)",
     min(len(c) for c in all_fixed),
     min(len(c) for c in all_recursive)),
    ("Tamano max (chars)",
     max(len(c) for c in all_fixed),
     max(len(c) for c in all_recursive)),
    ("Desviacion estandar (chars)",
     f"{(sum((len(c) - sum(len(c) for c in all_fixed)/len(all_fixed))**2 for c in all_fixed) / len(all_fixed))**0.5:.0f}",
     f"{(sum((len(c) - sum(len(c) for c in all_recursive)/len(all_recursive))**2 for c in all_recursive) / len(all_recursive))**0.5:.0f}")
]

for nombre, fixed, recursive in metricas:
    print(f"{nombre:<35} {str(fixed):>15} {str(recursive):>15}")

print("=" * 70)

In [ ]:
# === Visualizacion: Distribucion de tamanos de chunks ===

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de tamanos - Fixed
axes[0].hist([len(c) for c in all_fixed], bins=20, color="#4A90D9", edgecolor="black", alpha=0.7)
axes[0].set_title("Fixed-Size Chunking", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Tamano del chunk (caracteres)")
axes[0].set_ylabel("Frecuencia")
axes[0].axvline(x=CHUNK_SIZE, color="red", linestyle="--", label=f"chunk_size={CHUNK_SIZE}")
axes[0].legend()

# Histograma de tamanos - Recursive
axes[1].hist([len(c) for c in all_recursive], bins=20, color="#50C878", edgecolor="black", alpha=0.7)
axes[1].set_title("Recursive Character Splitting", fontsize=14, fontweight="bold")
axes[1].set_xlabel("Tamano del chunk (caracteres)")
axes[1].set_ylabel("Frecuencia")
axes[1].axvline(x=CHUNK_SIZE, color="red", linestyle="--", label=f"chunk_size={CHUNK_SIZE}")
axes[1].legend()

plt.suptitle("Distribucion de tamanos de chunks por estrategia", fontsize=16, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

print("\n>>> El fixed-size genera chunks muy uniformes (todos ~500 chars, excepto el ultimo).")
print(">>> El recursive genera chunks mas variados porque respeta limites naturales del texto.")
print(">>> Para RAG, generalmente el recursive es preferible: chunks mas coherentes = mejores embeddings.")

**Decision**: Para el resto del notebook usaremos **Recursive Character Splitting** porque produce chunks mas coherentes semanticamente, lo cual mejora la calidad de los embeddings y, en consecuencia, la calidad del retrieval.

> **Trade-off clave**: El recursive splitting genera chunks de tamano variable. Esto significa que algunos chunks seran mas cortos (y potencialmente menos informativos) que otros. En produccion, es comun filtrar chunks por debajo de un tamano minimo.

---

## Paso 3: Crear Embeddings e Indexar en Vector Store

Ahora convertimos cada chunk en un **vector de alta dimension** (embedding) y lo almacenamos en ChromaDB.

### Por que ChromaDB?
- Open-source y ligero (corre in-memory o persistido en disco)
- API simple y Pythonica
- Soporta metadata filtering (crucial para RAG en produccion)
- Ideal para prototipos y proyectos educativos

### El modelo de embedding
Usaremos `text-embedding-3-small` de OpenAI:
- **1536 dimensiones** de salida
- **Costo**: ~$0.02 por 1M de tokens
- **Rendimiento**: Excelente para la mayoria de casos de uso

In [ ]:
# === Paso 3: Crear embeddings e indexar ===

# Modelo de embedding a utilizar
EMBEDDING_MODEL = "text-embedding-3-small"

def crear_embedding(texto: str) -> list[float]:
    """
    Genera un embedding para un texto dado usando la API de OpenAI.
    
    Args:
        texto: El texto a convertir en vector.
    
    Returns:
        Lista de floats representando el embedding.
    """
    respuesta = client.embeddings.create(
        input=texto,
        model=EMBEDDING_MODEL
    )
    return respuesta.data[0].embedding


def crear_embeddings_batch(textos: list[str]) -> list[list[float]]:
    """
    Genera embeddings para multiples textos en una sola llamada a la API.
    Esto es mas eficiente que llamar uno por uno.
    
    Args:
        textos: Lista de textos a convertir.
    
    Returns:
        Lista de embeddings (cada uno es una lista de floats).
    """
    respuesta = client.embeddings.create(
        input=textos,
        model=EMBEDDING_MODEL
    )
    # Ordenar por indice para mantener el orden original
    return [item.embedding for item in sorted(respuesta.data, key=lambda x: x.index)]


# --- Inicializar ChromaDB (in-memory para este notebook) ---
chroma_client = chromadb.Client()

# Crear (o recrear) la coleccion
COLLECTION_NAME = "novatech_knowledge_base"

# Eliminar coleccion existente si la hay (para re-ejecuciones limpias)
try:
    chroma_client.delete_collection(name=COLLECTION_NAME)
except Exception:
    pass

collection = chroma_client.create_collection(
    name=COLLECTION_NAME,
    metadata={"hnsw:space": "cosine"}  # Distancia coseno para similitud
)

print(f"Coleccion '{COLLECTION_NAME}' creada en ChromaDB.")
print(f"Metrica de distancia: cosine")

# --- Indexar chunks con embeddings y metadata ---
total_indexados = 0

for nombre_archivo, chunks in recursive_chunks_por_doc.items():
    domain = archivos_fuente[nombre_archivo]["domain"]
    
    print(f"\nIndexando {nombre_archivo} ({len(chunks)} chunks)...")
    
    # Generar embeddings en batch (mas eficiente)
    embeddings = crear_embeddings_batch(chunks)
    
    # Preparar datos para ChromaDB
    ids = [f"{domain}_chunk_{i}" for i in range(len(chunks))]
    metadatas = [
        {
            "source": nombre_archivo,
            "chunk_id": i,
            "domain": domain,
            "chunk_size": len(chunk)
        }
        for i, chunk in enumerate(chunks)
    ]
    
    # Insertar en ChromaDB
    collection.add(
        ids=ids,
        embeddings=embeddings,
        documents=chunks,
        metadatas=metadatas
    )
    
    total_indexados += len(chunks)
    print(f"  -> {len(chunks)} chunks indexados correctamente.")

print(f"\n{'=' * 60}")
print(f"INDEXACION COMPLETA")
print(f"Total documentos en la coleccion: {collection.count()}")
print(f"{'=' * 60}")

In [ ]:
# === Inspeccionar una entrada de ejemplo ===

# Obtener un documento de ejemplo con su metadata
muestra = collection.get(
    ids=["productos_chunk_0"],
    include=["documents", "metadatas", "embeddings"]
)

print("EJEMPLO DE ENTRADA EN EL VECTOR STORE")
print("=" * 60)
print(f"ID: {muestra['ids'][0]}")
print(f"\nMetadata:")
for key, value in muestra['metadatas'][0].items():
    print(f"  {key}: {value}")
print(f"\nDocumento (texto):")
print(f"  {muestra['documents'][0][:200]}...")
print(f"\nEmbedding (primeros 10 valores de {len(muestra['embeddings'][0])} dimensiones):")
print(f"  {muestra['embeddings'][0][:10]}")
print("=" * 60)

---

## Paso 4: Retrieval - Encontrar Informacion Relevante

El **retrieval** es el corazon del pipeline RAG. Dado un query del usuario, buscamos los chunks mas semanticamente similares en nuestro vector store.

### Como funciona?
1. El query del usuario se convierte a un embedding (usando el mismo modelo)
2. ChromaDB calcula la distancia coseno entre el query embedding y todos los chunks almacenados
3. Retorna los top-K chunks mas cercanos (menor distancia = mayor similitud)

### Filtros por metadata
Ademas de la similitud semantica, podemos filtrar por metadata. Por ejemplo:
- Solo buscar en documentos del dominio "tecnica"
- Solo buscar en chunks de un archivo especifico

Esto es **extremadamente util** en produccion para mejorar precision.

In [ ]:
# === Paso 4: Funcion de Retrieval ===

def retrieve(
    query: str,
    n_results: int = 3,
    domain_filter: Optional[str] = None
) -> dict:
    """
    Recupera los chunks mas relevantes para una consulta dada.
    
    Args:
        query: La pregunta o consulta del usuario.
        n_results: Numero de chunks a recuperar.
        domain_filter: Filtrar por dominio ('productos' o 'tecnica').
                       None = buscar en todos.
    
    Returns:
        Dict con keys: 'documents', 'metadatas', 'distances', 'ids'
    """
    # Construir filtro de metadata (si aplica)
    where_filter = None
    if domain_filter:
        where_filter = {"domain": domain_filter}
    
    # Generar embedding del query
    query_embedding = crear_embedding(query)
    
    # Buscar en ChromaDB
    resultados = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        where=where_filter,
        include=["documents", "metadatas", "distances"]
    )
    
    return {
        "documents": resultados["documents"][0],
        "metadatas": resultados["metadatas"][0],
        "distances": resultados["distances"][0],
        "ids": resultados["ids"][0]
    }


def mostrar_resultados_retrieval(query: str, resultados: dict) -> None:
    """Helper para imprimir resultados de retrieval de forma legible."""
    print(f"\nQUERY: \"{query}\"")
    print("=" * 70)
    
    for i, (doc, meta, dist) in enumerate(zip(
        resultados["documents"],
        resultados["metadatas"],
        resultados["distances"]
    )):
        similitud = 1 - dist  # Convertir distancia coseno a similitud
        print(f"\n--- Resultado #{i+1} (similitud: {similitud:.4f}) ---")
        print(f"    Fuente: {meta['source']} | Dominio: {meta['domain']} | Chunk ID: {meta['chunk_id']}")
        print(f"    Texto ({len(doc)} chars):")
        # Mostrar texto con wrapping para legibilidad
        for linea in textwrap.wrap(doc, width=80):
            print(f"      {linea}")
    
    print("\n" + "=" * 70)


print("Funcion de retrieval definida. Probemos con 3 consultas diversas.")

In [ ]:
# === Probar retrieval con 3 consultas diversas ===

consultas_test = [
    ("cuanto cuesta Analytics Pro?", None),
    ("como hago rollback de un deploy?", None),
    ("que planes tiene el AI Assistant?", None),
]

for query, domain in consultas_test:
    resultados = retrieve(query, n_results=3, domain_filter=domain)
    mostrar_resultados_retrieval(query, resultados)

In [ ]:
# === Ejemplo de retrieval con filtro por dominio ===

print("\n" + "*" * 70)
print("EJEMPLO: Misma consulta, filtrada por dominio")
print("*" * 70)

query_filtro = "como se manejan las actualizaciones?"

# Sin filtro
print("\n[SIN FILTRO]")
res_sin_filtro = retrieve(query_filtro, n_results=2)
for meta in res_sin_filtro["metadatas"]:
    print(f"  -> Dominio: {meta['domain']} | Fuente: {meta['source']}")

# Solo tecnica
print("\n[FILTRO: domain='tecnica']")
res_tecnica = retrieve(query_filtro, n_results=2, domain_filter="tecnica")
for meta in res_tecnica["metadatas"]:
    print(f"  -> Dominio: {meta['domain']} | Fuente: {meta['source']}")

print("\n>>> Los filtros por metadata son cruciales cuando tienes multiples dominios.")
print(">>> Permiten dirigir la busqueda al contexto correcto y reducir ruido.")

---

## Paso 5: Generacion con Contexto (RAG Generation)

Ahora conectamos el retrieval con un LLM. La idea es simple pero poderosa:

1. Recuperamos los chunks relevantes (Paso 4)
2. Los inyectamos como **contexto** en el prompt del LLM
3. Le indicamos al LLM que responda **unicamente** basandose en ese contexto

### El system prompt es critico

Un buen system prompt para RAG debe:
- Instruir al modelo a usar SOLO el contexto proporcionado
- Pedir que cite las fuentes
- Indicar que diga "no tengo informacion" si el contexto no es suficiente
- Definir el tono y formato de la respuesta

In [ ]:
# === Paso 5: Generacion con contexto ===

# Modelo de generacion
GENERATION_MODEL = "gpt-4o-mini"

# System prompt para RAG - este es uno de los elementos mas importantes
RAG_SYSTEM_PROMPT = """Eres un asistente experto de NovaTech Solutions. Tu trabajo es responder 
preguntas de forma precisa y util, basandote UNICAMENTE en el contexto proporcionado.

REGLAS ESTRICTAS:
1. Responde SOLO con informacion presente en el contexto. No inventes ni asumas.
2. Si el contexto no contiene informacion suficiente para responder, di: 
   "No tengo informacion suficiente en mi base de conocimiento para responder esta pregunta."
3. Cita la fuente del contexto cuando sea posible (nombre del archivo fuente).
4. Se conciso pero completo. Usa listas cuando sea apropiado.
5. Si la pregunta es ambigua, indica las posibles interpretaciones.

FORMATO DE RESPUESTA:
- Responde en espanol
- Usa markdown para formato
- Al final, indica las fuentes usadas
"""


def generate_answer(query: str, context: str) -> str:
    """
    Genera una respuesta usando el LLM con el contexto proporcionado.
    
    Args:
        query: La pregunta del usuario.
        context: El contexto recuperado del vector store (chunks concatenados).
    
    Returns:
        La respuesta generada por el LLM.
    """
    # Construir el user prompt con el contexto
    user_prompt = f"""CONTEXTO (informacion de la base de conocimiento):
---
{context}
---

PREGUNTA DEL USUARIO:
{query}

Responde basandote unicamente en el contexto proporcionado arriba."""
    
    respuesta = client.chat.completions.create(
        model=GENERATION_MODEL,
        messages=[
            {"role": "system", "content": RAG_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1,  # Baja temperatura para respuestas mas factuales
        max_tokens=1024
    )
    
    return respuesta.choices[0].message.content


print("Funcion de generacion definida.")
print(f"Modelo: {GENERATION_MODEL}")
print(f"Temperature: 0.1 (baja, para respuestas factuales)")

In [ ]:
# === Probar generacion con las mismas 3 consultas ===

consultas_generacion = [
    "cuanto cuesta Analytics Pro?",
    "como hago rollback de un deploy?",
    "que planes tiene el AI Assistant?"
]

for query in consultas_generacion:
    print(f"\n{'#' * 70}")
    print(f"QUERY: \"{query}\"")
    print(f"{'#' * 70}")
    
    # 1. Retrieval
    resultados = retrieve(query, n_results=3)
    
    # 2. Construir contexto concatenando chunks recuperados
    context = "\n\n".join([
        f"[Fuente: {meta['source']}]\n{doc}" 
        for doc, meta in zip(resultados["documents"], resultados["metadatas"])
    ])
    
    # 3. Mostrar el prompt completo (transparencia)
    print(f"\n--- CONTEXTO ENVIADO AL LLM ({len(context)} chars) ---")
    print(context[:500] + "..." if len(context) > 500 else context)
    
    # 4. Generar respuesta
    respuesta = generate_answer(query, context)
    
    print(f"\n--- RESPUESTA DEL LLM ---")
    print(respuesta)
    print(f"{'#' * 70}")

---

## Pipeline Completo: Encadenando Todo

Ahora encapsulamos todo el flujo (retrieve + generate) en una sola funcion que representa nuestro **pipeline RAG completo**.

Esta es la funcion que expondrias como API en produccion.

In [ ]:
# === Pipeline RAG Completo ===

def rag_pipeline(
    query: str,
    n_results: int = 3,
    domain_filter: Optional[str] = None
) -> dict:
    """
    Pipeline RAG completo: Retrieval + Generation.
    
    Flujo:
        1. Recibe query del usuario
        2. Recupera los chunks mas relevantes del vector store
        3. Construye contexto con los chunks y sus fuentes
        4. Genera respuesta con el LLM usando el contexto
        5. Retorna respuesta estructurada con trazabilidad
    
    Args:
        query: Pregunta del usuario.
        n_results: Numero de chunks a recuperar.
        domain_filter: Filtrar por dominio (opcional).
    
    Returns:
        Diccionario con: query, retrieved_chunks, sources, answer, model.
    """
    # --- Paso 1: Retrieval ---
    resultados_retrieval = retrieve(
        query=query,
        n_results=n_results,
        domain_filter=domain_filter
    )
    
    # --- Paso 2: Construir contexto ---
    context = "\n\n".join([
        f"[Fuente: {meta['source']}]\n{doc}"
        for doc, meta in zip(
            resultados_retrieval["documents"],
            resultados_retrieval["metadatas"]
        )
    ])
    
    # --- Paso 3: Generar respuesta ---
    answer = generate_answer(query=query, context=context)
    
    # --- Paso 4: Construir respuesta estructurada ---
    # Extraer fuentes unicas
    sources = list(set(
        meta["source"] for meta in resultados_retrieval["metadatas"]
    ))
    
    return {
        "query": query,
        "retrieved_chunks": [
            {
                "text": doc[:200] + "..." if len(doc) > 200 else doc,
                "source": meta["source"],
                "domain": meta["domain"],
                "similarity": round(1 - dist, 4)
            }
            for doc, meta, dist in zip(
                resultados_retrieval["documents"],
                resultados_retrieval["metadatas"],
                resultados_retrieval["distances"]
            )
        ],
        "sources": sources,
        "answer": answer,
        "model": GENERATION_MODEL
    }


def pretty_print_resultado(resultado: dict) -> None:
    """Imprime el resultado del pipeline de forma legible."""
    print(f"\n{'=' * 70}")
    print(f"PREGUNTA: {resultado['query']}")
    print(f"{'=' * 70}")
    
    print(f"\nChunks recuperados ({len(resultado['retrieved_chunks'])}):")
    for i, chunk in enumerate(resultado['retrieved_chunks']):
        print(f"  [{i+1}] Similitud: {chunk['similarity']:.4f} | "
              f"Dominio: {chunk['domain']} | Fuente: {chunk['source']}")
    
    print(f"\nFuentes consultadas: {', '.join(resultado['sources'])}")
    print(f"Modelo: {resultado['model']}")
    
    print(f"\n--- RESPUESTA ---")
    print(resultado['answer'])
    print(f"{'=' * 70}")


print("Pipeline RAG completo definido. Listo para pruebas.")

In [ ]:
# === Probar el pipeline completo con 5 consultas diversas ===

consultas_pipeline: list[str] = [
    "cuanto cuesta Analytics Pro?",
    "como hago rollback de un deploy?",
    "que planes tiene el AI Assistant?",
    "cuales son los tiempos de respuesta de soporte?",
    "que hago si me sale el error 'Pipeline timeout after 300s'?"
]

# Almacenar resultados para uso posterior
resultados_pipeline: list[dict] = []

for query in consultas_pipeline:
    resultado = rag_pipeline(query)
    resultados_pipeline.append(resultado)
    pretty_print_resultado(resultado)

---

## Evaluacion Basica de Calidad

Un pipeline RAG no esta completo sin **evaluacion**. En produccion necesitas medir sistematicamente la calidad.

### Dimensiones clave de evaluacion

| Dimension | Pregunta | Que mide |
|:----------|:---------|:---------|
| **Faithfulness** | La respuesta es fiel al contexto recuperado? | Detecta alucinaciones |
| **Relevance** | La respuesta es relevante a la pregunta? | Detecta divagaciones |
| **Groundedness** | La respuesta cita sus fuentes? | Detecta afirmaciones sin respaldo |
| **Coverage** | La respuesta cubre toda la informacion del contexto? | Detecta omisiones |

### Evaluacion automatica vs manual
- **Manual**: Humanos evaluan las respuestas (gold standard, pero costoso)
- **Automatica**: Otro LLM evalua las respuestas (escalable, pero imperfecto)
- **Hibrida**: LLM pre-filtra, humanos revisan casos dudosos

Aqui implementaremos una evaluacion basica: verificamos si la respuesta cita fuentes y si responde correctamente ante preguntas fuera de alcance.

In [ ]:
# === Evaluacion basica de calidad ===

def evaluar_respuesta(resultado: dict) -> dict:
    """
    Evaluacion basica de una respuesta del pipeline RAG.
    
    Verifica:
    - Si la respuesta menciona fuentes
    - Si la respuesta tiene contenido sustancial
    - Si la respuesta admite limitaciones cuando corresponde
    
    Args:
        resultado: Salida del rag_pipeline().
    
    Returns:
        Dict con metricas de evaluacion.
    """
    answer = resultado["answer"].lower()
    
    # Verificar si cita fuentes
    cita_fuentes = any(
        indicador in answer
        for indicador in ["fuente", "base_conocimiento", "segun", "de acuerdo", ".md"]
    )
    
    # Verificar si tiene contenido sustancial (no solo "no se")
    es_sustancial = len(answer) > 50
    
    # Verificar si reconoce limitaciones cuando no tiene info
    reconoce_limites = any(
        frase in answer
        for frase in ["no tengo informacion", "no dispongo", "no encuentro", 
                      "no cuento con", "fuera de mi", "no puedo responder"]
    )
    
    # Calcular similitud promedio de los chunks recuperados
    avg_similarity = sum(
        c["similarity"] for c in resultado["retrieved_chunks"]
    ) / len(resultado["retrieved_chunks"])
    
    return {
        "query": resultado["query"],
        "cita_fuentes": cita_fuentes,
        "es_sustancial": es_sustancial,
        "reconoce_limites": reconoce_limites,
        "avg_retrieval_similarity": round(avg_similarity, 4),
        "answer_length": len(resultado["answer"])
    }


# --- Test 1: Consulta donde el contexto ES relevante ---
print("TEST 1: Consulta con contexto relevante")
print("=" * 60)

resultado_relevante = rag_pipeline("cuales son los planes y precios de DataSync?")
eval_relevante = evaluar_respuesta(resultado_relevante)

print(f"Query: {eval_relevante['query']}")
print(f"Cita fuentes: {eval_relevante['cita_fuentes']}")
print(f"Es sustancial: {eval_relevante['es_sustancial']}")
print(f"Similitud promedio retrieval: {eval_relevante['avg_retrieval_similarity']}")
print(f"Longitud respuesta: {eval_relevante['answer_length']} chars")
print(f"\nRespuesta:\n{resultado_relevante['answer']}")

In [ ]:
# --- Test 2: Consulta donde el contexto NO es relevante ---
print("\nTEST 2: Consulta FUERA del alcance de la base de conocimiento")
print("=" * 60)

resultado_fuera = rag_pipeline("cual es la capital de Francia?")
eval_fuera = evaluar_respuesta(resultado_fuera)

print(f"Query: {eval_fuera['query']}")
print(f"Cita fuentes: {eval_fuera['cita_fuentes']}")
print(f"Reconoce limites: {eval_fuera['reconoce_limites']}")
print(f"Similitud promedio retrieval: {eval_fuera['avg_retrieval_similarity']}")
print(f"Longitud respuesta: {eval_fuera['answer_length']} chars")
print(f"\nRespuesta:\n{resultado_fuera['answer']}")

print("\n" + "=" * 60)
print("ANALISIS:")
print("-" * 60)
print("- Un buen sistema RAG debe RECHAZAR preguntas fuera de alcance.")
print("- Observa la similitud promedio del retrieval: si es baja, el contexto")
print("  probablemente no es relevante y el LLM deberia indicarlo.")
print("- En produccion, podemos agregar un THRESHOLD de similitud minima")
print("  para decidir si el contexto recuperado es suficientemente relevante.")

---

## RAG Naive vs RAG con Contexto

Esta es la comparacion mas importante del notebook. Vamos a enviar la **misma pregunta** de dos formas:

1. **Directo al LLM** (sin contexto, sin RAG) -- el LLM solo puede usar su conocimiento parametrico
2. **A traves del pipeline RAG** -- el LLM tiene acceso a nuestra base de conocimiento

Esto demuestra el valor fundamental de RAG: **respuestas fundamentadas vs respuestas potencialmente alucinadas**.

In [ ]:
# === Comparacion: LLM Naive vs RAG Pipeline ===

def respuesta_naive(query: str) -> str:
    """
    Envia la pregunta directamente al LLM sin ningun contexto.
    Simula lo que pasa cuando NO usas RAG.
    """
    respuesta = client.chat.completions.create(
        model=GENERATION_MODEL,
        messages=[
            {
                "role": "system",
                "content": "Eres un asistente de NovaTech Solutions. Responde las preguntas del usuario."
            },
            {"role": "user", "content": query}
        ],
        temperature=0.1,
        max_tokens=1024
    )
    return respuesta.choices[0].message.content


# --- Comparacion lado a lado ---
query_comparacion = "cuales son los planes y precios de NovaTech AI Assistant y que integraciones soporta?"

print(f"QUERY: \"{query_comparacion}\"")
print("\n" + "=" * 70)

# Respuesta Naive (sin RAG)
print("\n[1] RESPUESTA NAIVE (LLM sin contexto)")
print("-" * 50)
resp_naive = respuesta_naive(query_comparacion)
print(resp_naive)

print("\n" + "=" * 70)

# Respuesta RAG
print("\n[2] RESPUESTA RAG (LLM + contexto recuperado)")
print("-" * 50)
resultado_rag = rag_pipeline(query_comparacion)
print(resultado_rag["answer"])

print("\n" + "=" * 70)
print("\nDIFERENCIAS CLAVE:")
print("-" * 50)
print("[Naive] El LLM INVENTA datos que no conoce (hallucination).")
print("        Los precios, features y detalles pueden ser incorrectos.")
print("        No hay forma de verificar la informacion.")
print("")
print("[RAG]   El LLM responde con datos REALES de la base de conocimiento.")
print("        Los precios y features son verificables contra la fuente.")
print("        La respuesta esta fundamentada y cita sus fuentes.")
print("=" * 70)

---

## Visualizacion: Flujo del Pipeline

Creemos una representacion visual del pipeline para consolidar lo aprendido.

In [ ]:
# === Visualizacion del Pipeline RAG ===

fig, ax = plt.subplots(figsize=(16, 8))
ax.set_xlim(0, 16)
ax.set_ylim(0, 10)
ax.axis("off")
ax.set_title("Pipeline RAG Completo - NovaTech Knowledge Base", 
             fontsize=18, fontweight="bold", pad=20)

# --- Fase de Indexacion (arriba) ---
ax.text(8, 9.3, "FASE DE INDEXACION (offline)", ha="center", fontsize=13, 
        fontweight="bold", color="#1a5276",
        bbox=dict(boxstyle="round,pad=0.3", facecolor="#d4e6f1", edgecolor="#1a5276"))

# Cajas de la fase de indexacion
index_steps = [
    (1.5, 7.5, "Documentos\n(.md files)", "#f9e79f"),
    (5.0, 7.5, "Chunking\n(Recursive)", "#abebc6"),
    (8.5, 7.5, "Embeddings\n(OpenAI)", "#aed6f1"),
    (12.0, 7.5, "Vector Store\n(ChromaDB)", "#d7bde2"),
]

for x, y, label, color in index_steps:
    rect = mpatches.FancyBboxPatch((x - 1.2, y - 0.6), 2.4, 1.2,
                                    boxstyle="round,pad=0.15",
                                    facecolor=color, edgecolor="black", linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x, y, label, ha="center", va="center", fontsize=10, fontweight="bold")

# Flechas de indexacion
for i in range(len(index_steps) - 1):
    x_start = index_steps[i][0] + 1.3
    x_end = index_steps[i + 1][0] - 1.3
    ax.annotate("", xy=(x_end, 7.5), xytext=(x_start, 7.5),
                arrowprops=dict(arrowstyle="->", lw=2, color="#2c3e50"))

# --- Fase de Consulta (abajo) ---
ax.text(8, 5.3, "FASE DE CONSULTA (online, por cada query)", ha="center", fontsize=13, 
        fontweight="bold", color="#7b241c",
        bbox=dict(boxstyle="round,pad=0.3", facecolor="#fadbd8", edgecolor="#7b241c"))

# Cajas de la fase de consulta
query_steps = [
    (1.5, 3.5, "Query\n(usuario)", "#fad7a0"),
    (5.0, 3.5, "Retrieval\n(top-k)", "#a9dfbf"),
    (8.5, 3.5, "Context\nAssembly", "#a9cce3"),
    (12.0, 3.5, "LLM\n(GPT-4o-mini)", "#d2b4de"),
]

for x, y, label, color in query_steps:
    rect = mpatches.FancyBboxPatch((x - 1.2, y - 0.6), 2.4, 1.2,
                                    boxstyle="round,pad=0.15",
                                    facecolor=color, edgecolor="black", linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x, y, label, ha="center", va="center", fontsize=10, fontweight="bold")

# Flechas de consulta
for i in range(len(query_steps) - 1):
    x_start = query_steps[i][0] + 1.3
    x_end = query_steps[i + 1][0] - 1.3
    ax.annotate("", xy=(x_end, 3.5), xytext=(x_start, 3.5),
                arrowprops=dict(arrowstyle="->", lw=2, color="#7b241c"))

# Flecha de Vector Store a Retrieval (conexion entre fases)
ax.annotate("", xy=(5.0, 4.2), xytext=(12.0, 6.8),
            arrowprops=dict(arrowstyle="->", lw=2, color="#8e44ad", linestyle="dashed"))
ax.text(8.8, 5.8, "busqueda\nsimilitud", ha="center", fontsize=9, 
        fontstyle="italic", color="#8e44ad")

# Caja de respuesta final
rect_resp = mpatches.FancyBboxPatch((10.8, 1.2), 2.4, 1.0,
                                     boxstyle="round,pad=0.15",
                                     facecolor="#82e0aa", edgecolor="#1e8449", linewidth=2)
ax.add_patch(rect_resp)
ax.text(12.0, 1.7, "Respuesta\nFundamentada", ha="center", va="center", 
        fontsize=10, fontweight="bold", color="#1e8449")

# Flecha de LLM a respuesta
ax.annotate("", xy=(12.0, 2.2), xytext=(12.0, 2.8),
            arrowprops=dict(arrowstyle="->", lw=2, color="#1e8449"))

plt.tight_layout()
plt.show()

print("\nEl diagrama muestra las dos fases del pipeline RAG:")
print("  1. INDEXACION (arriba): Se ejecuta una vez al cargar documentos.")
print("  2. CONSULTA (abajo): Se ejecuta por cada pregunta del usuario.")
print("  La linea punteada conecta el Vector Store con el Retrieval.")

---

## Trade-offs y Consideraciones de Produccion

### 1. Analisis de Costos

| Operacion | Modelo | Costo aproximado | Cuando se ejecuta |
|:----------|:-------|:-----------------|:-------------------|
| Embedding de chunks | `text-embedding-3-small` | ~$0.02 / 1M tokens | Una vez (indexacion) |
| Embedding del query | `text-embedding-3-small` | ~$0.02 / 1M tokens | Cada consulta |
| Generacion (input) | `gpt-4o-mini` | ~$0.15 / 1M tokens | Cada consulta |
| Generacion (output) | `gpt-4o-mini` | ~$0.60 / 1M tokens | Cada consulta |

> **Conclusion**: Los embeddings son extremadamente baratos. El costo principal esta en la **generacion**, especialmente los tokens de output. Optimizar el tamano del contexto reduce costos significativamente.

### 2. Latencia por Paso

| Paso | Latencia tipica | Optimizacion |
|:-----|:----------------|:-------------|
| Embedding del query | 50-100ms | Batch si hay multiples queries |
| Busqueda en vector store | 5-50ms | Indices HNSW, filtros pre-query |
| Generacion LLM | 500ms-3s | Streaming, modelos mas pequenos |
| **Total** | **~600ms - 3.5s** | |

> La generacion domina la latencia. Usa **streaming** para mejorar la percepcion del usuario.

### 3. Cuando cachear embeddings

- **Siempre** para los documentos fuente (no cambian frecuentemente)
- **Considerar** para queries frecuentes (cache LRU por query normalizado)
- **No cachear** el output del LLM si la temperatura > 0 (respuestas no deterministas)

### 4. Cuando la estrategia de chunking importa mas

- **Documentos largos y heterogeneos**: El chunking es critico. Un mal chunk mezcla temas.
- **Documentos cortos y homogeneos**: El chunking importa menos.
- **Tablas y datos estructurados**: Requieren chunking especializado (no romper filas).
- **Codigo fuente**: Usar tree-sitter o AST-aware chunking.
- **Regla general**: Si tus resultados de retrieval mezclan temas no relacionados, tu chunking necesita mejoras.

---

## Errores Comunes en Pipelines RAG

### 1. Chunks demasiado grandes o demasiado pequenos
- **Demasiado grandes** (>1000 chars): Traen mucho ruido al contexto. El LLM se distrae con informacion irrelevante. Ademas, los embeddings de textos largos tienden a ser "promedios" poco informativos.
- **Demasiado pequenos** (<100 chars): Pierden contexto necesario. Un chunk que dice "$99/mes" sin decir a que producto se refiere es inutil.
- **Regla practica**: 200-600 caracteres es un buen rango inicial. Ajustar segun los resultados de evaluacion.

### 2. No incluir metadata en los chunks
- Sin metadata, no puedes filtrar por dominio, fecha, autor, etc.
- En produccion, la metadata es lo que permite construir experiencias como "buscar solo en documentacion tecnica" o "solo documentos del ultimo mes".
- **Siempre** almacena al menos: fuente, fecha de creacion, dominio/categoria.

### 3. No evaluar la calidad del retrieval antes de optimizar la generacion
- Si el retrieval trae chunks irrelevantes, no importa que tan bueno sea tu prompt o tu LLM.
- **Primero** asegura que el retrieval funcione bien. **Despues** optimiza la generacion.
- Usa metricas como Precision@K, Recall@K y MRR (Mean Reciprocal Rank).

### 4. Ignorar el system prompt
- Un system prompt debil permite que el LLM alucine incluso con buen contexto.
- **Siempre** incluye instrucciones explicitas de: usar solo el contexto, citar fuentes, admitir cuando no sabe.
- El system prompt es codigo: iteralo, versionalo y evalualo como cualquier otro componente.

### 5. No manejar el caso "fuera de alcance"
- Cuando un usuario pregunta algo que no esta en tu base de conocimiento, el sistema debe responder honestamente.
- **Implementa un threshold de similitud**: Si el mejor chunk tiene similitud < 0.3, probablemente no hay informacion relevante.
- **No fuerces respuestas**: Es mejor decir "no se" que inventar una respuesta incorrecta.

---

## Checklist de Comprension

Antes de continuar al siguiente notebook, asegurate de poder responder estas preguntas:

- [ ] **1.** Por que el **Recursive Character Splitting** generalmente produce mejores resultados que el Fixed-Size Chunking para RAG? Que trade-off introduce?

- [ ] **2.** Si aumentas el `chunk_size` de 500 a 2000 caracteres, que impacto esperarias en (a) la calidad del retrieval, (b) el costo de embeddings y (c) la calidad de la respuesta del LLM?

- [ ] **3.** Explica por que el **system prompt** del LLM es tan importante en un pipeline RAG. Que pasa si no le dices "responde solo con el contexto"?

- [ ] **4.** En la comparacion RAG vs Naive, por que el LLM "naive" puede dar respuestas que *parecen* correctas pero son alucinaciones? Como verificarias esto en produccion?

- [ ] **5.** Disena una estrategia para manejar el caso donde el usuario pregunta algo que NO esta en la base de conocimiento. Que componentes del pipeline modificarias?

---

### Siguiente paso

En el **Notebook 03** exploraremos tecnicas avanzadas de RAG:
- Hybrid search (keyword + semantic)
- Re-ranking de resultados
- Multi-query retrieval
- Evaluacion sistematica con RAGAS framework